In [ ]:
# https://guide.ncloud-docs.com/docs/clovastudio-2-2-1#

In [1]:
# -*- coding: utf-8 -*-
import collections
from naver_ai import NaverAI 

naver = NaverAI()

In [2]:
request_data = {
    "maxTokens": 32,
    "temperature": 0.5,
    "topK": 0,
    "topP": 0.8,
    "repeatPenalty": 5.0,
    "restart": "",
    "includeTokens": True,
    "includeAiFilters": True,
    "includeProbs": False
}

In [9]:
def make_content(original_dict: dict, new_dict: dict):
    return dict(collections.ChainMap(original_dict, new_dict))

## Text Generation

In [4]:
preset_text = "한국어: 사과\n영어: Apple\n한국어: 바나나"
start = "\n영어"
stopBefore = ["\n한국어"]

text_gen_req = make_content(request_data, keys, [preset_text, start, stopBefore])

In [5]:
generation_result = naver.execute(text_gen_req)
generation_result

{'status': {'code': '20000', 'message': 'OK', 'messageVariables': {}},
 'result': {'aiFilter': [{'groupName': 'curse',
    'name': 'insult',
    'score': '2'},
   {'groupName': 'curse', 'name': 'discrimination', 'score': '2'},
   {'groupName': 'unsafeContents', 'name': 'sexualHarassment', 'score': '2'}],
  'text': '한국어: 사과\n영어: Apple\n한국어: 바나나\n영어: Banana',
  'stopReason': 'stop_before',
  'inputLength': 15,
  'outputLength': 7,
  'inputTokens': ['한국',
   '어',
   ':',
   ' 사과',
   '\n',
   '영어',
   ':',
   ' Apple',
   '\n',
   '한국',
   '어',
   ':',
   ' 바나나',
   '\n',
   '영어'],
  'outputTokens': [':', ' B', 'an', 'ana', '\n', '한국', '어'],
  'probs': [],
  'ok': True}}

In [54]:
text = """문장: 그녀는 이 상황이 부끄러웠다. 그녀가 밖으로 달아났다.
묘사: 그녀는 연신 몸을 움찔거리며 주위를 살폈다.
그녀의 어깨는 딱딱하게 굳어 위로 솟아 있었고 얼굴은 발갛게 달아올라 있었다.
주변의 시선을 의식할 때마다 광활한 광장에 발가벗고 서 있는 듯한 부끄러움이 밀려왔다.
방에서 뛰쳐나가고 싶었지만 누군가 다리를 도끼로 내려친 듯이 한 발자국도 움직일 수 없었다.
###
문장: 사랑에 빠지다.
"""
novel_req = {"text": text,
            "stopBefore": ["###\n", "문장:", "묘사:"],
            "start": "\n묘사:",
            "restart": ["\n###", "\n문장:"],
            "maxToken": 800}

novel_req = make_content(request_data, novel_req)

In [55]:
novel_res = naver.execute(novel_req)
novel_res["result"]["text"]

'문장: 그녀는 이 상황이 부끄러웠다. 그녀가 밖으로 달아났다.\n묘사: 그녀는 연신 몸을 움찔거리며 주위를 살폈다.\n그녀의 어깨는 딱딱하게 굳어 위로 솟아 있었고 얼굴은 발갛게 달아올라 있었다.\n주변의 시선을 의식할 때마다 광활한 광장에 발가벗고 서 있는 듯한 부끄러움이 밀려왔다.\n방에서 뛰쳐나가고 싶었지만 누군가 다리를 도끼로 내려친 듯이 한 발자국도 움직일 수 없었다.\n###\n문장: 사랑에 빠지다.\n\n묘사: 그는 마치 자석처럼 그녀에게 끌렸다.\n그는 자신의 감정을 주체할 수 없어 계속해서 그녀를 따라다녔다.\n자신과 같은'

## Transformation

In [63]:
text = '문장: 너랑 걔네들은 다르잖아. 너대로 하면되\n교정: 너랑 걔네들은 다르잖아. 너대로 하면 돼\n'

transform_req = {"text": text,
 "stopBefore": ['###', '문장:', '교정:', '###\n'], 
 "start": "\n교정",
 "restart": "\n###\n문장:"}

transform_req = make_content(request_data, transform_req)

In [64]:
transform_result = naver.execute(transform_req)
transform_result["result"]["text"]

'문장: 너랑 걔네들은 다르잖아. 너대로 하면되\n교정: 너랑 걔네들은 다르잖아. 너대로 하면 돼\n\n교정: (핸드폰을 들고) 여보세요? 어, 그래. 나야. 응. 알았어. 금방 갈게. 고마워. 아~ 정말 다행이다. 그치?'

## Classification

In [15]:
req_contents = {"text": '문장: "기분 진짜 좋다"\n감정: 긍정\n###\n문장: "아오 진짜 짜증나게 하네"\n감정: 부정\n###\n문장: "하 힘들다 진짜 쉬고싶다"',
                "start": "\n감정",
                "stopBefore": ["###", "문장: ", "감정:", "###\n"]}

classification_req = make_content(request_data, req_contents)

In [16]:
classification_res = naver.execute(classification_req)

In [19]:
classification_res["result"]["text"]

'문장: "기분 진짜 좋다"\n감정: 긍정\n###\n문장: "아오 진짜 짜증나게 하네"\n감정: 부정\n###\n문장: "하 힘들다 진짜 쉬고싶다"\n감정: 부정\n'

## QA 

In [24]:
text = """질문: ESFJ는 문제에 봉착했을때 어떻게 대응하는가?
답: 현실적인 해결 방법을 찾기 위해 노력한다.
###
질문: ISFJ는 연인에게 어떻게 대하는 편인가?
답: 섬세하고 다정하게 케어해주는 편이다.
####
질문: INTP는 사람들이 많은 곳에 가면 어떻게 행동하는가?
답: 주변의 상황을 파악하기 위해 관찰하는 편이다.
###
질문: ESFT는 충동적인 선택을 많이 하는 편인가?"""

req_contents = {"text": text,
               "start": "\n답",
                "stopBefore": ["###", "질문: ", "답:", "###\n"],
                "maxToken": 50}

req_contents = make_content(request_data, req_contents)

In [26]:
qa_res = naver.execute(req_contents)
qa_res["result"]["text"]

'질문: ESFJ는 문제에 봉착했을때 어떻게 대응하는가?\n답: 현실적인 해결 방법을 찾기 위해 노력한다.\n###\n질문: ISFJ는 연인에게 어떻게 대하는 편인가?\n답: 섬세하고 다정하게 케어해주는 편이다.\n####\n질문: INTP는 사람들이 많은 곳에 가면 어떻게 행동하는가?\n답: 주변의 상황을 파악하기 위해 관찰하는 편이다.\n###\n질문: ESFT는 충동적인 선택을 많이 하는 편인가?\n답: 아니다. 계획적으로 움직이는 편이다.\n'

## Summarization

In [36]:
text = """줄거리: "미나리는 어디서든 잘 자라" 낯선 미국, 아칸소로 떠나온 한국 가족. 가족들에게 뭔가 해내는 걸 보여주고 싶은 아빠 '제이콥'(스티븐 연)은 자신만의 농장을 가꾸기 시작하고 엄마 '모니카'(한예리)도 다시 일자리를 찾는다. 아직 어린 아이들을 위해 ‘모니카’의 엄마 ‘순자’(윤여정)가 함께 살기로 하고 가방 가득 고춧가루, 멸치, 한약 그리고 미나리씨를 담은 할머니가 도착한다. 의젓한 큰딸 '앤'(노엘 케이트 조)과 장난꾸러기 막내아들 '데이빗'(앨런 김)은 여느 그랜마같지 않은 할머니가 영- 못마땅한데… 함께 있다면, 새로 시작할 수 있다는 희망으로 하루하루 뿌리 내리며 살아가는 어느 가족의 아주 특별한 여정이 시작된다!"""

req_sum = {"text": text, 
           "start": "\n한줄요약",
          "stopBefore": ["###\n", "줄거리:", "한줄요약:"],
          "restart": ["\n###", "\n줄거리:"], 
          "maxToken": 150}
req_sum = make_content(request_data, req_sum)

In [37]:
sum_req = naver.execute(req_sum)

In [38]:
sum_req["result"]["text"]

'줄거리: "미나리는 어디서든 잘 자라" 낯선 미국, 아칸소로 떠나온 한국 가족. 가족들에게 뭔가 해내는 걸 보여주고 싶은 아빠 \'제이콥\'(스티븐 연)은 자신만의 농장을 가꾸기 시작하고 엄마 \'모니카\'(한예리)도 다시 일자리를 찾는다. 아직 어린 아이들을 위해 ‘모니카’의 엄마 ‘순자’(윤여정)가 함께 살기로 하고 가방 가득 고춧가루, 멸치, 한약 그리고 미나리씨를 담은 할머니가 도착한다. 의젓한 큰딸 \'앤\'(노엘 케이트 조)과 장난꾸러기 막내아들 \'데이빗\'(앨런 김)은 여느 그랜마같지 않은 할머니가 영- 못마땅한데… 함께 있다면, 새로 시작할 수 있다는 희망으로 하루하루 뿌리 내리며 살아가는 어느 가족의 아주 특별한 여정이 시작된다!\n한줄요약: 잔잔하게 흘러가는 영화지만 가슴에 와닿는 대사들이 많다.\n영화 <미나리> 후기\n<미나리>는 1980년대 아메리칸드림을 꿈꾸며'